---
title: Linux高级系统编程 chapter6 高级IO函数 pipe 和 splice
tags: 小书匠,c++,socket,io,pipe,splice,Books-Linux高级系统编程 
grammar_cjkRuby: true
renderNumberedHeading: true
---

[toc]

# Linux高级系统编程 chapter6 高级IO函数 pipe 和 splice

## pipe

### 原型

```cpp
#include <unistd.h>
int pipe(int fd[2]);
```

### 参数

参数 fd 是一个长度为 2 的整数，分别代表输出管道和输入管道。从 `fd[1]` 输入，并从 `fd[0]` 中读出。

### 返回值

- 成功时，返回0，并将文件描述符写入 fd 中。
- 失败时，返回-1，并设置 errno

### splice

## 示例

### 使用 pipe 和 splice 来实现回声服务器

- 在 macOS Mojave 10.14.6 系统下测试，无法运行，会报错。因为 macOS 上没有 splice 函数。

In [42]:
%%file server.cpp
#include <iostream>
#include <cstring> // for memset
#include <sys/socket.h>
#include <arpa/inet.h> // inet_addr
#include <netinet/in.h> // IPPROTO_TCP
#include <unistd.h>
#include <errno.h>
#include <fcntl.h>
#include <assert.h>

#define BUFF_SIZE 100

int main() {
    int serv_sock = socket(AF_INET, SOCK_STREAM, IPPROTO_TCP);

    // 定义 serv 地址
    sockaddr_in serv_addr;
    memset(&serv_addr, 0, sizeof(serv_addr));
    serv_addr.sin_family = AF_INET; // 使用 IPv4
    serv_addr.sin_addr.s_addr = inet_addr("127.0.0.1"); //指定 ip
    serv_addr.sin_port = htons(1234); // 指定端口

    bind(serv_sock, (sockaddr*)&serv_addr, sizeof(serv_addr));

    listen(serv_sock, 20);

    // 定义 client 地址
    sockaddr_in client_addr;
    socklen_t client_addr_size = sizeof(client_addr);

    int client_sock = accept(serv_sock, (sockaddr*)&client_addr, &client_addr_size);
    printf("[Server] Client %s conect to the server!\n", inet_ntoa(client_addr.sin_addr));

    int pipefd[2];
    int ret = pipe(pipefd);
    if (ret == -1) 
    {
        perror("[Server] Create pipefd failed");
        close(serv_sock);
        close(client_sock);
        return 0;
    }

    int iret = 0;
    while (1)
    {
        iret = splice(client_sock, NULL, pipefd[1], NULL, 32768, 0);
        if (iret != -1)
        {
            printf("[Server] recieve %d bytes from client\n", iret);
            iret = splice(pipefd[0], NULL, client_sock, NULL, 32768, 0);
            if (iret != -1)
            {
                printf("[Server] send %d bytes to client\n", iret);
            }
            else 
            {
                perror("[Server] Send error");
                break;
            }
        }
        else
        {
            perror("[Server] Splice failed");
            break;
        }
    }
    close(client_sock);
    close(serv_sock);
    return 0;
}

Overwriting server.cpp


In [44]:
%%file client.cpp
#include <cstdio>
#include <unistd.h>
#include <arpa/inet.h>
#include <sys/socket.h>
#include <cstring>
#include <iostream>
#include <errno.h>

#define BUFF_SIZE 1024

using namespace std;

int main()
{

    int serv_sock = socket(AF_INET, SOCK_STREAM, IPPROTO_TCP);

    sockaddr_in serv_addr;
    memset(&serv_addr, 0, sizeof(serv_addr));
    serv_addr.sin_family = AF_INET;
    serv_addr.sin_addr.s_addr = inet_addr("127.0.0.1");
    serv_addr.sin_port = htons(1234);

    int iret;
    iret = connect(serv_sock, (sockaddr*)&serv_addr, sizeof(serv_addr)); // 连接
    if ( iret == -1)  // 判断是否连接成功
    {
        perror("Connect to client failed!");
        close(serv_sock);
        return 0;
    }

    // 获取用户输入
    char send_buffer[BUFF_SIZE];
    char recv_buffer[BUFF_SIZE];

    // 发送给服务器
    while (true)
    {
        printf("[Client] Input a string: ");
        scanf("%s", send_buffer);

        iret = send(serv_sock, send_buffer, sizeof(send_buffer), 0);
        if (iret != -1) {
            printf("[Client] Message to server: %s\n", send_buffer);
        } else {
            perror("[Client] send failed");
        }

        iret = recv(serv_sock, recv_buffer, sizeof(recv_buffer), 0);
        if (iret > 0) {
            printf("[Client] Message from server: %s\n", recv_buffer);
        } else 
        {
            if (iret == 0)
                printf("[Client] server closed!\n");
            else
                perror("[Client] connection failed");
            break;
        }
        memset(send_buffer, 0, sizeof(send_buffer));
        memset(recv_buffer, 0, sizeof(recv_buffer));
    }
    close(serv_sock);
    return 0;
}

Overwriting client.cpp


In [45]:
!g++ server.cpp -o server

server.cpp:47:16: error: use of undeclared identifier 'splice'
        iret = splice(client_sock, NULL, pipefd[1], NULL, 32768, 0);
               ^
server.cpp:51:20: error: use of undeclared identifier 'splice'
            iret = splice(pipefd[0], NULL, client_sock, NULL, 32768, 0);
                   ^
2 errors generated.


In [46]:
!g++ client.cpp -o client